In [1]:
from dipy.reconst import mapmri
from dipy.viz import fvtk
from dipy.data import fetch_cenir_multib, read_cenir_multib, get_sphere
from dipy.core.gradients import gradient_table
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

Download and read the data for this tutorial.

MAPMRI requires multi-shell data, to properly fit the radial part of the basis.

In [ ]:
with open('../config.json', 'r') as json_file:
    conf = json.load(json_file)

subject_path = conf['HCP']['data_paths']['mgh_1007']

In [ ]:

small_data = data[45:55, 45:55, 40:50, :]
np.save('{}/mri/small_data.npy'.format(subject_path), small_data)

For this example we select only the shell with b-values equal to the one of the
Human Connectome Project (HCP).

For the values of the q-space
indices to make sense it is necessary to explicitly state the big_delta and
small_delta parameters in the gradient table.

In [3]:
bvals = [1000, 2000, 3000]
img, gtab = read_cenir_multib(bvals)
big_delta = 0.0365  # seconds
small_delta = 0.0157  # seconds
gtab = gradient_table(bvals=gtab.bvals, bvecs=gtab.bvecs,
                      small_delta=big_delta,
                      big_delta=small_delta)
data = img.get_data()
data_small = data[40:65, 50:51, 35:60]

print('data.shape (%d, %d, %d, %d)' % data.shape)

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\sesjojen\.dipy\cenir_multib 
data.shape (104, 104, 72, 612)


Instanstiate the MAPMRI Model.

In [4]:
radial_order = 6
map_model_laplacian_aniso = mapmri.MapmriModel(gtab, radial_order=radial_order,
                                               laplacian_regularization=True,
                                               laplacian_weighting=.2)

We can then fit the MAPMRI model to the data.

In [5]:
mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(data_small)

### Compute the ODFs

Load an odf reconstruction sphere

In [6]:
sphere = get_sphere('symmetric724')

The radial order s can be increased to sharpen the results, but it might
also make the odfs noisier. Always check the results visually.

In [7]:
odf = mapfit_laplacian_aniso.odf(sphere, s=2)
print('odf.shape (%d, %d, %d, %d)' % odf.shape)

odf.shape (25, 1, 25, 724)


Display the ODFs

In [8]:
r = fvtk.ren()
sfu = fvtk.sphere_funcs(odf, sphere, colormap='jet')
sfu.RotateX(-90)
fvtk.add(r, sfu)
fvtk.record(r, n_frames=1, out_path='odfs.png', size=(600, 600))